# Data check

[Simon Dobson](mailto:simon.dobson@st-andrews.ac.uk) <br>
School of Computer Science, University of St Andrews, Scotland UK

This notebook is intended to check the rain gauge datasets downloaded and created from SEPA and CEDA. It also downloads recent data from the EPA live API.

In [1]:
from io import BytesIO
import json
from datetime import date, datetime, timedelta
import netCDF4 as nc
from geopandas import GeoDataFrame
from PIL import Image

from sensor_placement.data import stations, uk_epa

import logging
logging.basicConfig(level=logging.DEBUG)

from pyproj import CRS, Transformer
from shapely.geometry import Point, shape
import folium

In [2]:
uk_grid_crs = CRS.from_string('EPSG:27700')   # UK national grid
latlon_crs = CRS.from_string('EPSG:4326')     # global Mercator (WGS 84)

proj = Transformer.from_crs(uk_grid_crs, latlon_crs)

## County boundaries

In [3]:
boundaries_filename = 'datasets/UK_BUC.geojson'
with open(boundaries_filename, 'r') as fh:
    counties_json = json.load(fh)

In [4]:
counties = GeoDataFrame({'county': map(lambda c: c['properties']['ctyua18nm'], counties_json['features']),
                         'geometry': map(lambda c: shape(c['geometry']), counties_json['features'])})

We check the boundaries for validity (although there isn't a lot we can do about it).

In [5]:
counties[counties.geometry.is_valid == False]

INFO:shapely.geos:Ring Self-intersection at or near point -0.75933436735766302 51.103258672002717


,county,geometry
134,Hampshire,"MULTIPOLYGON (((-0.76891 51.10327, -0.75944 51..."


## SEPA rain gauges and a sample month

A sample of SEPA datra should have been created during installation by ``make datasets``.

In [6]:
sepa_filename = 'datasets/sepa_monthly_2017.nc'
sepa = nc.Dataset(sepa_filename)

In [7]:
sepastations = stations(sepa)

/home/sd80/programming/sensor-placement/venv3/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [8]:
sepastations

,name,east,north,longitude,latitude,geometry
id,,,,,,
15018,Abbey St Bathans,375000,662000,-2.387448,55.853298,POINT (-2.38745 55.85330)
234150,Aberlour,327000,843000,-3.205975,57.480217,POINT (-3.20598 57.48022)
115301,Affric Lodge,218000,823000,-5.005980,57.262726,POINT (-5.00598 57.26273)
234170,Alford,356000,817000,-2.717123,57.242302,POINT (-2.71712 57.24230)
115302,Allanfearn,271000,847000,-4.151564,57.499149,POINT (-4.15156 57.49915)
...,...,...,...,...,...,...
115660,Waulkmill Glen,252000,658000,-4.353628,55.794842,POINT (-4.35363 55.79484)
116008,Weisdale Mill,439000,1153000,-1.288604,60.259327,POINT (-1.28860 60.25933)
115250,Westhill,382000,806000,-2.291989,57.147568,POINT (-2.29199 57.14757)


## CEDA rain gauges and a sample month 

Similarly, there should be sample CEDA data available after ``make datasets``.

In [9]:
ceda_filename = 'datasets/ceda_midas_monthly_2017.nc'
ceda = nc.Dataset(ceda_filename)

In [10]:
cedastations = stations(ceda)

/home/sd80/programming/sensor-placement/venv3/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [11]:
cedastations

,name,east,north,longitude,latitude,geometry
id,,,,,,
38,achfary,229000,939000,-4.916,58.311001,POINT (-4.91600 58.31100)
8231,huntsham,299000,118000,-3.438,50.952999,POINT (-3.43800 50.95300)
50,strathy-east,284000,965000,-3.992,58.561001,POINT (-3.99200 58.56100)
60,poolewe,186000,881000,-5.597,57.773998,POINT (-5.59700 57.77400)
64,plockton,180000,833000,-5.653,57.337002,POINT (-5.65300 57.33700)
...,...,...,...,...,...,...
24219,mannington-hall,614000,332000,1.179,52.844002,POINT (1.17900 52.84400)
57118,bute-rothesay-no2,208000,663000,-5.056,55.825001,POINT (-5.05600 55.82500)
57233,margam-no-2,280000,185000,-3.721,51.555000,POINT (-3.72100 51.55500)


## UK EPA live rainfall

By contrast, there's no default EPA data downloaded, so we do this "live" here. This can be quite time-consuming, as it involves retrieving data from all 980+ stations using the REST API.

The data isn't saved, it's simply sucked into an in-memory dataset.

In [12]:
today = datetime.now()
start = today - timedelta(days=2)

epa = uk_epa(start, today, None)

DEBUG:root:Retrieving list of stations from http://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): environment.data.gov.uk:80
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/stations?parameter=rainfall HTTP/1.1" 200 None
DEBUG:root:No location information for Rainfall station -- ignored
DEBUG:root:No location information for Rainfall station -- ignored
DEBUG:root:No location information for Rainfall station -- ignored
DEBUG:root:No location information for Rainfall station -- ignored
DEBUG:root:Retrieving readings for station E7050 from http://environment.data.gov.uk/flood-monitoring/id/measures/E7050-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/E7050-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate

DEBUG:root:Retrieving readings for station E24238 from http://environment.data.gov.uk/flood-monitoring/id/measures/E24238-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/E24238-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station E10510 from http://environment.data.gov.uk/flood-monitoring/id/measures/E10510-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/E10510-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 000181TP from http://environment.data.gov.uk/flood-monitoring/id/measures/000181TP-ra

DEBUG:root:Retrieving readings for station 037292 from http://environment.data.gov.uk/flood-monitoring/id/measures/037292-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/037292-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 086575 from http://environment.data.gov.uk/flood-monitoring/id/measures/086575-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/086575-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station E1179 from http://environment.data.gov.uk/flood-monitoring/id/measures/E1179-rainfall

DEBUG:root:Retrieving readings for station E7045 from http://environment.data.gov.uk/flood-monitoring/id/measures/E7045-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/E7045-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 44200 from http://environment.data.gov.uk/flood-monitoring/id/measures/44200-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/44200-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 058461 from http://environment.data.gov.uk/flood-monitoring/id/measures/058461-rainfall-tip

DEBUG:root:Retrieving readings for station 47167 from http://environment.data.gov.uk/flood-monitoring/id/measures/47167-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/47167-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 277407TP from http://environment.data.gov.uk/flood-monitoring/id/measures/277407TP-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/277407TP-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station E1815 from http://environment.data.gov.uk/flood-monitoring/id/measures/E1815-rainf

...

DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/582970-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 48103 from http://environment.data.gov.uk/flood-monitoring/id/measures/48103-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/48103-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 49147 from http://environment.data.gov.uk/flood-monitoring/id/measures/49147-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/49147-rainfall-tipping_bucket_raing

DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/577794-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 4527 from http://environment.data.gov.uk/flood-monitoring/id/measures/4527-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/4527-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 3642 from http://environment.data.gov.uk/flood-monitoring/id/measures/3642-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/3642-rainfall-tipping_bucket_raingauge-t

DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/1750-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 1810 from http://environment.data.gov.uk/flood-monitoring/id/measures/1810-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/1810-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 1408 from http://environment.data.gov.uk/flood-monitoring/id/measures/1408-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/1408-rainfall-tipping_bucket_raingauge-t-1

DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/019271-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station E42297 from http://environment.data.gov.uk/flood-monitoring/id/measures/E42297-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/E42297-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 1930 from http://environment.data.gov.uk/flood-monitoring/id/measures/1930-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/1930-rainfall-tipping_bucket_raing

DEBUG:root:Retrieving readings for station 015313 from http://environment.data.gov.uk/flood-monitoring/id/measures/015313-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/015313-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station NelsonGC from http://environment.data.gov.uk/flood-monitoring/id/measures/NelsonGC-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13
DEBUG:urllib3.connectionpool:http://environment.data.gov.uk:80 "GET /flood-monitoring/id/measures/NelsonGC-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?startdate=2022-04-11&enddate=2022-04-13 HTTP/1.1" 200 None
DEBUG:root:Retrieving readings for station 575092 from http://environment.data.gov.uk/flood-monitoring/id/measures/575092-

In [13]:
epastations = stations(epa)

/home/sd80/programming/sensor-placement/venv3/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


## The datasets

We can now display all the stations we have access to on an interactive map.

In [14]:
uk_gauges = folium.Map(location=(55, -3), tiles="Stamen Terrain", zoom_start=5)

# add the boundaries
county_boundaries_layer = folium.FeatureGroup(name='County boundaries')
for _, r in counties.iterrows():
    folium.GeoJson(r['geometry']).add_to(county_boundaries_layer)
county_boundaries_layer.add_to(uk_gauges)
    
# add the SEPA stations
sepa_layer = folium.FeatureGroup(name='SEPA stations')
for i in sepastations.index:
    s = sepastations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='blue', icon='cloud')).add_to(sepa_layer)
sepa_layer.add_to(uk_gauges)

# add the CEDA stations
ceda_layer = folium.FeatureGroup(name='CEDA MIDAS stations')
for i in cedastations.index:
    s = cedastations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='green', icon='cloud')).add_to(ceda_layer)
ceda_layer.add_to(uk_gauges)

# add the EPA live stations
epa_layer = folium.FeatureGroup(name='EPA stations')
for i in epastations.index:
    s = epastations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='red', icon='cloud')).add_to(epa_layer)
epa_layer.add_to(uk_gauges)
    
# add a layer countrol
_ = folium.LayerControl().add_to(uk_gauges)

In [15]:
uk_gauges

In [16]:
img_data = uk_gauges._to_png(5)
img = Image.open(BytesIO(img_data))
img.save('datasets/epa-sepa-ceda.png')

DEBUG:selenium.webdriver.remote.remote_connection:POST http://localhost:45185/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "firefox", "acceptInsecureCerts": true, "moz:debuggerAddress": true, "pageLoadStrategy": "normal", "moz:firefoxOptions": {"args": ["--headless"]}}}, "desiredCapabilities": {"browserName": "firefox", "acceptInsecureCerts": true, "moz:debuggerAddress": true, "pageLoadStrategy": "normal", "moz:firefoxOptions": {"args": ["--headless"]}}}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:45185
DEBUG:urllib3.connectionpool:http://localhost:45185 "POST /session HTTP/1.1" 200 734
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://localhost:45185/session/8ef46d8f-3c90-4c47-a00d-db79107dc5a5/url {"url": "file:////tmp/folium_5tod0ezv.html"}
DEBUG:urllib3.connectionpool:http://localhost:45185 "POST /session/8ef46d8f-3c90-4c47-a00d-db79107dc5a5/ur